In [ ]:
from dreimac import CircularCoords

from circle_bundles.viz.pca_viz import show_pca
from circle_bundles.viz.thumb_grids import *
from circle_bundles.viz.fiber_vis import *
from circle_bundles.viz.nerve_vis import *
from circle_bundles.viz.base_vis import *
from circle_bundles.viz.lattice_vis import *
from circle_bundles.viz.circle_vis import *
from circle_bundles.bundle import build_bundle, attach_bundle_viz_methods
from circle_bundles.covers import MetricBallCover
from circle_bundles.metrics import RP1_dist2, get_dist_mat
from circle_bundles.local_triv import *
attach_bundle_viz_methods()

from synthetic.opt_flow_patches import *
from optical_flow.patch_viz import *


#For saving
folder_path = '/Users/bradturow/Desktop/Diagrams/Paper Diagrams/'

# Optical Flow Torus Model

In [ ]:
#Generate a sampling of the optical flow torus model over RP1
n_samples = 5000
n = 3
data, base_points, alpha = sample_opt_flow_torus(n_samples, dim = n)

print(f'{n_samples} optical flow patches generated.')

In [ ]:
#View a sample of the data
patch_vis = make_patch_visualizer()
fig = show_data_vis(data, patch_vis, sampling_method = None, max_samples = 30)
plt.show()

In [ ]:
#Show a PCA visualization of the dataset
show_pca(data)

In [ ]:
#Construct a cover of the base space
n_landmarks = 12
lmk_angs = np.linspace(0, np.pi,n_landmarks, endpoint= False)
landmarks = np.array([np.cos(lmk_angs), np.sin(lmk_angs)]).T
overlap = 1.99

radius = overlap* np.pi/(2*n_landmarks)

cover = MetricBallCover(base_points, landmarks, radius, metric = RP1_dist2)
cover_data = cover.build()
print('Cover constructed.')

In [ ]:
summ = cover.summarize(plot = True)

In [ ]:
#Construct local circular coordinates and model transitions as O(2) matrices
bundle = build_bundle(
    data,
    cover,
    CircularCoords_cls=CircularCoords,
    show=True,
)


In [ ]:
#Compute class persistence on the weights filtration of the nerve
pers = bundle.get_persistence(show = True)


In [ ]:
#Get a global coordinatization compatible with the maximal subcomplex of the nerve on which 
#the characteristic class representatives are coboundaries 

triv_result = bundle.get_global_trivialization()
print('Global coordinates computed.')

In [ ]:
#Show an interactive visualization of the bundle colored by fiber coordinate
fig = bundle.show_bundle(get_dist_mat = get_dist_mat, colors = triv_result.F)
plt.show()

In [ ]:
#Show the correlations between local circular coordinates on overlaps
fig = bundle.compare_trivs()
plt.show()

In [ ]:
#Show a visualization of the nerve labeled with SW1

#Compute a potential for the restricted orientation class
subcomplex = bundle.get_max_trivial_subcomplex()
edges = subcomplex.kept_edges
Omega = bundle.classes.cocycle_used.restrict(edges)
phi_vec = Omega.orient_if_possible(edges)[2]
phi = {lmk: phi_vec[lmk] for lmk in range(n_landmarks)}
omega = bundle.classes.omega_O1_used

fig = bundle.show_circle_nerve(omega = omega, phi = phi)
plt.show()

In [ ]:
#Show a recovered patch diagram
per_row = 5
per_col = 9
predom_dirs = np.arctan2(base_points[:,1], base_points[:,0]) % np.pi
coords = np.column_stack([predom_dirs.reshape(-1,1), triv_result.F.reshape(-1,1)])

fig = lattice_vis(
    data,
    coords,
    patch_vis,
    per_row=per_row,
    per_col = per_col,
    figsize=15,
    thumb_px=350,   
    dpi=350         
)

plt.show()
